# Подготовка данных для нейронной сети  
Используется соотношение открытого интереса по Call и Put опционам.  
Запись результатов в файл nn_features_and_target.csv

In [1]:
from pathlib import Path
import pandas as pd
import sqlite3
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import sys
import importlib

sys.dont_write_bytecode = True

In [2]:
tiker: str = 'RTS'
db_path: Path = Path(fr'c:\Users\Alkor\gd\data_quote_db\{tiker}_futures_options_day.db')

# Подключение к базе данных
conn = sqlite3.connect(db_path)

# Чтение данных из БД в DataFrame
df_f = pd.read_sql_query("SELECT `TRADEDATE`, `OPEN`, `LOW`, `HIGH`, `CLOSE`, `OPENPOSITION`, `SHORTNAME`, `LSTTRADE` FROM Futures", conn)
df_o = pd.read_sql_query("SELECT `TRADEDATE`, `OPENPOSITION`, `NAME`, `LSTTRADE`, `OPTIONTYPE`, `STRIKE` FROM Options", conn)

# Закрытие соединения
conn.close()

df_f[['TRADEDATE', 'LSTTRADE']] = df_f[['TRADEDATE', 'LSTTRADE']].apply(pd.to_datetime)
df_o[['TRADEDATE', 'LSTTRADE']] = df_o[['TRADEDATE', 'LSTTRADE']].apply(pd.to_datetime)

df_o = df_o[df_o.TRADEDATE < df_o.LSTTRADE]

# Сортировка по полю даты и сброс индекса
df_f = df_f.sort_values(by='TRADEDATE').reset_index(drop=True)
df_o = df_o.sort_values(by='TRADEDATE').reset_index(drop=True)

# Вывод DataFrame
# print(df_f)
# print(df_o)
print(df_f.to_string(max_rows=4, max_cols=20))
print(df_f.shape)
print('\n', df_o.to_string(max_rows=4, max_cols=10))
print(df_o.shape)

      TRADEDATE      OPEN       LOW      HIGH     CLOSE  OPENPOSITION SHORTNAME   LSTTRADE
0    2015-01-05   76930.0   72470.0   78980.0   74600.0        751996  RTS-3.15 2015-03-16
1    2015-01-06   74470.0   71200.0   74610.0   73480.0        731236  RTS-3.15 2015-03-16
...         ...       ...       ...       ...       ...           ...       ...        ...
2411 2024-08-15  105010.0  103320.0  105250.0  103440.0         73922  RTS-9.24 2024-09-19
2412 2024-08-16  103420.0  102710.0  104100.0  103010.0         77014  RTS-9.24 2024-09-19
(2413, 8)

         TRADEDATE  OPENPOSITION      NAME   LSTTRADE OPTIONTYPE  STRIKE
0      2015-01-05         18004  RTS-3.15 2015-01-15          C  100000
1      2015-01-05             0  RTS-3.15 2015-03-16          C   45000
...           ...           ...       ...        ...        ...     ...
768878 2024-08-16            14  RTS-9.24 2024-09-19          P  132500
768879 2024-08-16          1790  RTS-9.24 2024-09-19          P   97500
(768880, 6

In [3]:
step_strike = 2500

df_rez = pd.DataFrame()

for row in df_f.itertuples():  # Перебирает построчно DF с фьючерсами
    # Выбор из DF с опционами только опционов на дату фьючерса
    df = df_o[df_o.TRADEDATE == row.TRADEDATE]
    # print(df.to_string(max_rows=4, max_cols=10))
    
    # DF с опционами call
    df_p = (
        df.query('OPTIONTYPE ==  "P"')
        .groupby(['STRIKE'], as_index=False)
        .agg({'OPENPOSITION': 'sum'})
        .sort_values(['STRIKE'], ascending=True)
        .rename(columns={'OPENPOSITION': 'oi_p'})
    )
    # print(df_p.to_string(max_rows=4, max_cols=10))
    # print(df_p.shape)
    
    # DF с опционами put
    df_c = (
        df.query('OPTIONTYPE ==  "C"')
        .groupby(['STRIKE'], as_index=False)
        .agg({'OPENPOSITION': 'sum'})
        .sort_values(['STRIKE'], ascending=True)
        .rename(columns={'OPENPOSITION': 'oi_c'})
    )
    # print(df_c.to_string(max_rows=4, max_cols=10))
    # print(df_c.shape)
    
    # Временный DF со страйками (для последующего слияния), чтобы исключить пропуски страйков
    df_tmp = pd.DataFrame(columns=['STRIKE'])
    for st in range(df.STRIKE.min(), df.STRIKE.max() + step_strike, step_strike):
        # Новое значение для добавления
        new_row = pd.DataFrame({'STRIKE': [st]})
        # Добавление новой строки в DataFrame с помощью concat
        df_tmp = pd.concat([df_tmp, new_row], ignore_index=True)
    # print(df_tmp.to_string(max_rows=4, max_cols=10))
    
    # Выполнение полного внешнего соединения DF'ов с опционами call и put по столбцу STRIKE
    merged_df = pd.merge(df_p, df_c, on='STRIKE', how='outer')
    merged_df = pd.merge(merged_df, df_tmp, on='STRIKE', how='outer')
    # Приведение типов с помощью infer_objects
    merged_df = merged_df.infer_objects(copy=False)
    
    merged_df = merged_df.fillna(0)  # Пустые значения заполняем "0"
    # Приведение типов к int
    merged_df[['STRIKE', 'oi_c', 'oi_p']] = merged_df[['STRIKE', 'oi_c', 'oi_p']].astype(int)
    # print(merged_df.to_string(max_rows=4, max_cols=10))
    # print(merged_df.shape)
    
    # Переделываем колонку с ОИ по call в колонку с накопленной суммой ОИ
    merged_df['oi_c'] = merged_df['oi_c'].cumsum()
    # Переделываем колонку с ОИ по put в колонку с накопленной суммой ОИ в обратном порядке
    merged_df['oi_p'] = merged_df.iloc[::-1]['oi_p'].cumsum()[::-1]
    # print(merged_df.to_string(max_rows=4, max_cols=10))
    
    # Записываем в переменную дату торгов фьючерса
    trade_date = (df_f.loc[df_f['TRADEDATE'] == row.TRADEDATE, 'TRADEDATE'].values[0]).astype('datetime64[D]')
    # Записываем в переменную значение цены OPEN фьючерса
    price_open = df_f.loc[df_f['TRADEDATE'] == row.TRADEDATE, 'OPEN'].values[0]
    # Записываем в переменную значение цены CLOSE фьючерса
    price_close = df_f.loc[df_f['TRADEDATE'] == row.TRADEDATE, 'CLOSE'].values[0]
    # Записываем в переменную значение цены HIGH фьючерса
    price_high = df_f.loc[df_f['TRADEDATE'] == row.TRADEDATE, 'HIGH'].values[0]
    # Записываем в переменную значение цены LOW фьючерса
    price_low = df_f.loc[df_f['TRADEDATE'] == row.TRADEDATE, 'LOW'].values[0]
    # Расчитываем ближайший страйк к цене CLOSE
    nearest_strike = round(price_close / step_strike) * step_strike
    # print(f'{trade_date=}, {price_open=}, {price_close=}, {price_high=}, {price_low=}, {nearest_strike=}')
    
    # Список индексов со значением nearest_strike из поля merged_df['STRIKE']
    index_lst = merged_df.index[merged_df['STRIKE'] == nearest_strike].tolist()
    # Известный индекс строки (берем 0)
    index_nearest = index_lst[0]
    # Определение диапазона строк
    start_index = max(0, index_nearest - 10)
    end_index = min(len(df), index_nearest + 10 + 1)
    # Получение 10 строк до и 10 строк после строки с известным индексом ближайшего страйка
    subset_df = merged_df.iloc[start_index:end_index]
    subset_df = subset_df.copy()
    # Создание колонки соотношений (разницы) отрытого интереса Call и Put
    subset_df['oi'] = subset_df.apply(lambda x: x.oi_p - x.oi_c if x.STRIKE < price_close else x.oi_c - x.oi_p, axis=1)
    # print(subset_df.to_string(max_rows=40, max_cols=10))

    # Создаем объект MinMaxScaler
    scaler = MinMaxScaler()
    # Применяем нормализацию 0-1 к колонке oi
    subset_df['oi_norm'] = scaler.fit_transform(subset_df[['oi']])
    # print(subset_df.to_string(max_rows=40, max_cols=10))
    
    subset_df['strike_abc'] = subset_df['STRIKE'] - nearest_strike
    subset_df = subset_df.set_index('strike_abc')
    subset_df = subset_df.sort_index(ascending=True)  # ascending=False - сортировка по убываю
    subset_df = subset_df[['oi_norm']].T
    # Заменяем индекс строки на значение даты
    subset_df.index = [trade_date]
    # Заменяем название индекса
    subset_df = subset_df.rename_axis('date')
    # Добавление колонок
    subset_df['high_abc'] = price_high - nearest_strike
    subset_df['low_abc'] = price_low - nearest_strike
    subset_df['close_abc'] = price_close - nearest_strike
    subset_df['central_strike'] = nearest_strike
    subset_df['open'] = price_open
    subset_df['high'] = price_high
    subset_df['low'] = price_low
    subset_df['close'] = price_close
    # print(subset_df.to_string(max_rows=40, max_cols=20))
    
    #concatenate dataframes
    df_rez = pd.concat([df_rez, subset_df])
    
    # break

# Округляем значения в столбцах с 0 по 20 до 5 знаков после запятой
df_rez.iloc[:, 0:21] = df_rez.iloc[:, 0:21].round(6)
print(df_rez.to_string(max_rows=6, max_cols=20))
print(df_rez.shape)

strike_abc    -25000    -22500    -20000    -17500    -15000    -12500    -10000     -7500     -5000     -2500  ...     22500     25000  high_abc  low_abc  close_abc  central_strike      open      high       low     close
date                                                                                                            ...                                                                                                          
2015-01-05  1.000000  0.949765  0.946863  0.890311  0.886746  0.810817  0.768762  0.702327  0.658250  0.519462  ...  0.721658  0.876098    3980.0  -2530.0     -400.0           75000   76930.0   78980.0   72470.0   74600.0
2015-01-06  1.000000  0.999613  0.949249  0.946274  0.888331  0.884700  0.808215  0.764843  0.699029  0.650257  ...  0.737880  0.749382    2110.0  -1300.0      980.0           72500   74470.0   74610.0   71200.0   73480.0
2015-01-08  0.879807  0.815902  0.811820  0.731742  0.685201  0.616099  0.559507  0.401556  0.216736  0.112487  

In [4]:
# Сохранение DF в файл
df_rez.to_csv(fr'../nn_features_and_target.csv', index=True, sep=';')